In [1]:
import pandas as pd
import string
import re

# #from attr.filters import exclude
# from gradio.themes.builder_app import suggestions
# from tensorflow_io.python.ops.bigtable.bigtable_row_range import prefix


In [2]:
df = pd.read_csv(r"C:\Users\Asus\NLP\IMDB Dataset.csv\IMDB Dataset.csv")

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df['review'] = df['review'].str.lower()

In [5]:
def text_to_dict(filename, delimiter='='):
    my_dict = {}
    
    with open(filename, 'r') as file:
        for line in file:
            # Strip leading/trailing whitespace and split by the delimiter
            parts = line.strip().split(delimiter)
            
            # Check if we have exactly two parts after splitting
            if len(parts) == 2:
                key = parts[0].strip()
                value = parts[1].strip()
                my_dict[key] = value
    
    return my_dict

filename = r'C:\Users\Asus\NLP\IMDB Dataset.csv\slang.txt'
chat_words_dict = text_to_dict(filename)


In [6]:
def remove_urls(text):
    pattern = re.compile(r'https?://\S+|www\.\S+.*?>')
    return pattern.sub(r'',text)

def remove_html(text):
    pattern = re.compile(r'<.*?>')
    return pattern.sub(r'',text)

exclude = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('','',exclude))

def chat_word_treatment(text):
    arr = []
    for i in text.split():
        if i.upper() in chat_words_dict:
            arr.append(chat_words_dict[i.upper()])
        else:
            arr.append(i)
            
    return " ".join(arr)


In [7]:
def all_function(x):
    for func in function:
        x= func(x)
        return x

function = [remove_html,remove_urls,remove_punctuation,chat_word_treatment]
df['review']=df['review'].apply(all_function)

In [8]:
df.sample(5)

,review,sentiment
38277,"we don't have to lose this movie, this is one ...",positive
49636,i sat down through 2 hours of pure boredom. i ...,negative
4036,"why i disliked the movie, apart from the sheer...",negative
2967,this is like a school video project and a prop...,negative
14888,i watched this movie for its two hours and hav...,negative


# symspell (libabry) used for spelling correction!!!

In [9]:
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell(max_dictionary_edit_distance=2,prefix_length=7)
sym_spell.load_dictionary(r"C:\Users\Asus\NLP\twitter sentiment\symspelling_dict\frequency_dictionary_en_82_765.txt",term_index=0,count_index=0)

def correct_spell(sentence):
    words = sentence.split()
    
    corrected = []
    
    for word in words:
        
        suggestions = sym_spell.lookup(word,Verbosity.CLOSEST,max_edit_distance= 2)
        
        if suggestions:
            corrected.append(suggestions[0].term)
        else:
            corrected.append(word)
            
    return ' '.join(corrected)

In [10]:
df['review']=df['review'].apply(correct_spell)

In [11]:
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


# Stop words

In [12]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [14]:
def stop_words(texts):
    
    new_text=[]
    
    for word in texts.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    return " ".join(new_text)

        

In [15]:
df['review']=df['review'].apply(stop_words)